In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_excel('shihara_data.xlsx') 

In [8]:


def preprocess_payment_data(df):
    df = df.rename(columns={
        'Date': 'date',
        'Payments': 'payment',
        'Balance': 'balance'
    })

    # Remove commas and convert to float
    df['payment'] = pd.to_numeric(df['payment'].astype(str).str.replace(',', ''), errors='coerce')
    df['balance'] = pd.to_numeric(df['balance'].astype(str).str.replace(',', ''), errors='coerce')


    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    
    # Create daily payments summary
    daily_payments = df[df['payment'] > 0].groupby('date').agg(
        payment=('payment', 'sum'),
        payment_count=('payment', 'count')
    ).reset_index()
    
    # Create complete date range
    min_date = daily_payments['date'].min()
    max_date = daily_payments['date'].max()
    all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
    
    complete_df = pd.DataFrame({'date': all_dates})
    complete_df = complete_df.merge(daily_payments, on='date', how='left')
    complete_df['payment'] = complete_df['payment'].fillna(0)
    complete_df['payment_count'] = complete_df['payment_count'].fillna(0)
    complete_df = complete_df.sort_values('date').reset_index(drop=True)
    
    # Convert date to datetime for calculations
    complete_df['date'] = pd.to_datetime(complete_df['date'])
    
    # Calculate days since last payment
    days_since = []
    last_payment_date = None
    for i, row in complete_df.iterrows():
        current_date = row['date']
        if last_payment_date is None:
            days_since.append(0)
        else:
            days = (current_date - last_payment_date).days
            days_since.append(max(days, 1))
        if row['payment'] > 0:
            last_payment_date = current_date
    complete_df['days_since_last_payment'] = days_since
    
    # Create additional features
    complete_df['payment_made'] = (complete_df['payment'] > 0).astype(int)
    complete_df['7day_avg_payment'] = complete_df['payment'].rolling(window=7, min_periods=1).mean()
    complete_df['30day_avg_payment'] = complete_df['payment'].rolling(window=30, min_periods=1).mean()
    
    # Date-related features
    complete_df['day_of_week'] = complete_df['date'].dt.dayofweek
    complete_df['day_of_week_sin'] = np.sin(2 * np.pi * complete_df['day_of_week'] / 7)
    complete_df['day_of_week_cos'] = np.cos(2 * np.pi * complete_df['day_of_week'] / 7)
    complete_df['is_weekend'] = complete_df['day_of_week'].isin([5,6]).astype(int)
    complete_df = complete_df.drop(columns=['day_of_week'])
    
    complete_df['num_payments_from_start'] = complete_df['payment_made'].cumsum()
    
    # Add balance information
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    daily_balance = df.groupby('date')['balance'].last().reset_index()
    complete_df['date'] = complete_df['date'].dt.normalize()
    complete_df = complete_df.merge(daily_balance, on='date', how='left')
    complete_df['balance'] = complete_df['balance'].ffill()
    complete_df['balance_changed'] = (complete_df['balance'] != complete_df['balance'].shift(1)).astype(int)
    

    
    # Calculate cumulative average zero days
    cumulative_avg_zero_days = []
    total_zero_days = 0
    streak_count = 0
    in_streak = False

    for payment in complete_df['payment']:
        if payment == 0:
            total_zero_days += 1
            if not in_streak:
                streak_count += 1
                in_streak = True
        else:
            in_streak = False

        avg = round(total_zero_days / streak_count, 2) if streak_count > 0 else 0
        cumulative_avg_zero_days.append(avg)

    complete_df['cumulative_avg_zero_days'] = cumulative_avg_zero_days
    
    # Winsorize columns
    columns_to_winsorize = [
        'payment_count',
        'days_since_last_payment',
        '7day_avg_payment',
        '30day_avg_payment',
        'num_payments_from_start',
        'balance',
        'payment',
        'cumulative_avg_zero_days'
    ]
    
    def winsorize_column_iqr(df, column):
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
        return df
    
    for col in columns_to_winsorize:
        complete_df = winsorize_column_iqr(complete_df, col)
    
    # Normalize features
    exog_vars = [
        'payment_count',
        'days_since_last_payment',
        '7day_avg_payment',
        '30day_avg_payment',
        'cumulative_avg_zero_days',
        'num_payments_from_start', 
        'day_of_week_sin', 
        'day_of_week_cos', 
        'is_weekend', 
        'balance'
    ]

    
    for var in exog_vars:
        min_val = complete_df[var].min()
        max_val = complete_df[var].max()
        complete_df[var] = (complete_df[var] - min_val) / (max_val - min_val) if max_val != min_val else 0
    

    complete_df.ffill(inplace=True)
    
    
    
    return complete_df






In [9]:
complete_df = preprocess_payment_data(df)

In [10]:
complete_df.to_excel('shihara_processed_data.xlsx', index=False)
